In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Aula 5 - Parte 3

# Exercício 1
**Objetivo: ligar a decomposição por auto-valores e auto-vetores ao seu respectivo código computacional e verificar sua limitação para matrizes quadradas**

Já sabemos que uma matriz quadrada $X$ pode ser decomposta em termos de seus auto-vetores (na matriz $P$) e auto-valores (na diagonal da matriz $D$) na forma:

$$
A = P D P^{-1}.
$$

O código abaixo realiza essa decomposição e a recomposição.

a) Por que a matriz `B` no código é igual à matriz `A`?

b) Modifique o código para que a matriz `A` deixe de ser quadrada (por exemplo, adicionando uma linha ou coluna). Que erro é apontado no código?

c) Se a matriz $A$ na equação $A = P D P^{-1}$ não for quadrada, que problema deve aparecer na operação $P D P^{-1}$ em relação à dimensões de $P$ e $D$?


In [ ]:
A = np.array( [[1, 2], [2, 3]] )
w, v = np.linalg.eig(A)
#print(w, v)
w_ = np.diag(w)
#print(w_)
B = v @ w_ @ np.linalg.inv(v)

print(B)

# Exercício 2
**Objetivo: entender SVD como uma decomposição**

Quando calculamos $P$ e $D$ em $A=PDP^{-1}$, estamos *decompondo* a matriz $A$ em *componentes* ($P$, $D$ e $P^{-1}$).

Essa decomposição não é a única possibilidade. Uma outra, muito utilizada, é a decomposição em valores singulares (*Singular Value Decomposition*, ou **SVD**):

$$
A = U \Sigma V^T
$$

No código abaixo:

a) Confirme que a matriz `B` é igual à matriz `A`

b) Compare o conteúdo das matrizes `u`, `sigma` e `vt` ao conteúdo das matrizes `v` e `w_` no exercício anterior.

c) Altere o código para verificar se a decomposição SVD ainda funciona se a matriz `A` não for quadrada.

d) Se a matriz `A` não é quadrada, quais são as dimensões das matrizes `u`, `sigma` e `vt`?

e) O que a função `diagsvd` faz, e por que ela tem três parâmetros?

In [ ]:
from scipy.linalg import svd, diagsvd
import numpy as np
A = np.array( [[1, 2], [2, 3]] )
u, s, vt = svd(A)
sigma = diagsvd(s, A.shape[0], A.shape[1])

B = u @ sigma @ vt
print(A)
print(B)

# Exercício 3
**Objetivo: Manipular a matriz $\Sigma$ em uma imagem e verificar os resultados**

Na decomposição SVD, usamos a formulação:

$
A = U \Sigma V^T,
$

onde:

* As colunas de $U$ são os auto-vetores de $A^T A$,
* As colunas de $V$ (e, portanto, as linhas de $V^T$) são auto-vetores de $A A^T$,
* $\Sigma$ é uma matriz onde $s_{i,i}$ é a raiz quadrada dos auto-valores de $A^T A$ ou de $A A^T$.

A [demonstração disso está aqui](./demonstracao_svd.md), e uma outra perspectiva sobre essa demonstração está [aqui](https://gregorygundersen.com/blog/2018/12/20/svd-proof/).


Vamos agora verificar o que esses valores representam.

O código abaixo abre uma imagem em tons de cinza e faz uma decomposição SVD.

a) Qual é a dimensão da imagem lida?

b) Quais são as dimensões das matrizes `u`, `sigma` e `vt`?

c) Os valores da matriz `sigma` vêm da matriz `s`. O que acontece com a imagems se o primeiro elemento de `s` for modificado para ficar igual a `0` (dica: use `s[0] *= 0`)?

d) Remova progressivamente mais elementos da matriz `s` igualando-os a zero. O que acontece com a imagem formada?

e) Agora, remova elementos do *final* da matriz `s` igualando-os a zero. O que acontece com a imagem?

f) Experimente perceptualmente e verifique quantos elementos você consegue remover do final da matriz `s` até que perceba alguma diferença na imagem?

In [ ]:
from scipy.linalg import svd, diagsvd
import matplotlib.image as mpimg

A = mpimg.imread("rabbit.jpeg").mean(axis=2)[0::4,0::4]

u, s, vt = svd(A)

# Altere daqui para baixo
s = s
# Altere daqui para cima

sigma = diagsvd(s, A.shape[0], A.shape[1])

B = u @ sigma @ vt

plt.subplot(1,2,1)
plt.title('Imagem original')
plt.imshow(A, cmap='gray')
plt.subplot(1,2,2)
plt.title('Imagem alterada')
plt.imshow(B, cmap='gray')
plt.show()

# Exercício 4
**Objetivo: Remover ruídos de uma imagem manipulando a matriz S**

Uma possível maneira de remover ruídos de uma imagem é a seguinte:

* Fazemos a decomposiçao SVD da imagem
* Removemos os elementos de menor magnitude da matriz $\Sigma$
* Reconstituímos a imagem

a) Aplique esse procedimento para tentar remover os ruídos que foram adicionados à imagem abaixo.

b) Qual é a relação entre o ruído removido da imagem e a distorção causada na imagem?

In [ ]:
from scipy.linalg import svd, diagsvd
import matplotlib.image as mpimg

A = mpimg.imread("rabbit.jpeg").mean(axis=2)[0::4,0::4]
alpha = 0.3
A += alpha * np.random.random(A.shape) * np.max(A)

u, s, vt = svd(A)

# Altere daqui para baixo
s = s
# Altere daqui para cima

sigma = diagsvd(s, A.shape[0], A.shape[1])

B = u @ sigma @ vt


plt.subplot(1,2,1)
plt.title('Imagem ruidosa')
plt.imshow(A, cmap='gray')
plt.subplot(1,2,2)
plt.title('Imagem com ruído removido')
plt.imshow(B, cmap='gray')
plt.show()

# Exercício 5
**Objetivo: Comprimir uma imagem modificando a matriz $\Sigma$**

Muitas vezes, admitimos perdas em um processo de compressão. Isso acontece, por exemplo, em compressão de imagens ou de sons: uma pequena distorção, quase imperceptível, é aceitável, desde que isso possibilite comprimir dados.

A compressão de dados funciona "apagando" elementos de uma imagem ou de um som. As diferentes estratégias de compressão se diferenciam em *como escolher* os elementos que são mais imperceptíveis dentro de uma imagem ou som.

Nesta atividade, vamos fazer um compressor de imagens baseado em SVD. O método aplicado é bem conhecido, e uma boa explicação dele pode ser encontrada [aqui](https://medium.com/balabit-unsupervised/image-compression-using-singular-value-decomposition-de20451c69a3).

Veja que, quando eliminamos valores de $\Sigma$ fazendo-os ficar igual a zero, isso é equivalente a remover alguns elementos das matrizes $U$ e de $V^T$. Por exemplo, na decomposição:

$$
U \Sigma V^T = 
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 0.2
\end{bmatrix}
\begin{bmatrix}
4 & 3 \\
2 & 1
\end{bmatrix}
$$

Se eliminamos o valor $0.2$ de $\Sigma$, ficamos com:

$$ 
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 0
\end{bmatrix}
\begin{bmatrix}
4 & 3 \\
2 & 1
\end{bmatrix}

= 
\begin{bmatrix}
4 & 3 \\
12 & 9
\end{bmatrix}

$$

Isso é equivalente a simplemente remover esse valor de $\Sigma$, e remover as linhas e colunas de $U$ e $V^T$ que seriam multiplicadas por ele. Daí, ficamos com:
$$
\begin{bmatrix}
1  \\
3 
\end{bmatrix}
\begin{bmatrix}
1 
\end{bmatrix}
\begin{bmatrix}
4 & 3 \\
\end{bmatrix}
= 
\begin{bmatrix}
4 & 3 \\
12 & 9
\end{bmatrix}
$$

A vantagem de remover linhas e colunas diretamente é que passamos a precisar de menos espaço para armazenar nossos dados!

---

Analise o código abaixo. Ele é bastante parecido com o código acima, Altere a função `comprimir` para que ela realize a remoção de elementos que foi mostrada acima. Teste seu código!!!

a) Como o valor de $K$ influencia a compressão da imagem e a qualidade da imagem final?

b) Qual é o menor valor de $K$ para o qual a imagem não perde qualidade?

c) Qual é o maior valor de $K$ que permite reduzir o tamanho ocupado pela imagem?

In [ ]:
from scipy.linalg import svd, diagsvd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

# Altere esta função!
def comprimir (u, s, vt, K):
    """Remove elementos de u, s e vt deixando somente K componentes restantes
    """ 
    u_ = u
    s_ = s
    vt_ = vt
    return u_, s_, vt_

A = mpimg.imread("rabbit.jpeg")

u0, s0, vt0 = svd(A[:,:,0])
u1, s1, vt1 = svd(A[:,:,1])
u2, s2, vt2 = svd(A[:,:,2])

n_bytes_inicial = A.nbytes
print("Tamanho inicial da imagem (bytes): ", n_bytes_inicial)

K = 4

u0, s0, vt0 = comprimir(u0, s0, vt0, K)
u1, s1, vt1 = comprimir(u1, s1, vt1, K)
u2, s2, vt2 = comprimir(u2, s2, vt2, K)

sigma0 = diagsvd(s0, u0.shape[1], vt0.shape[0])
sigma1 = diagsvd(s1, u1.shape[1], vt1.shape[0])
sigma2 = diagsvd(s2, u2.shape[1], vt2.shape[0])

n_bytes_final = u0.nbytes + u1.nbytes + u2.nbytes + s0.nbytes + s1.nbytes + s2.nbytes + vt0.nbytes + vt1.nbytes + vt2.nbytes
print("Tamanho da representação comprimida: ", n_bytes_final)
print("Compressão:", n_bytes_final / n_bytes_inicial)

B = np.zeros_like(A)
B[:,:,0] = u0 @ sigma0 @ vt0
B[:,:,1] = u1 @ sigma1 @ vt1
B[:,:,2] = u2 @ sigma2 @ vt2

plt.subplot(1,2,1)
plt.title('Imagem original')
plt.imshow(A, cmap='gray')
plt.subplot(1,2,2)
plt.title('Imagem comprimida')
plt.imshow(B, cmap='gray')
plt.show()